In [ ]:
from bs4 import BeautifulSoup as bs
import geopy.distance
import numpy as np
import pandas as pd
import re
import requests

In [ ]:
url = 'https://en.wikipedia.org/wiki/User:Michael_J/County_table'
table_class = 'wikitable sortable jquery-tablesorter'
response = requests.get(url)
print(response)

soup = bs(response.text, 'html.parser')
table = soup.find('table',{'class':'wikitable'})

In [ ]:
table_df = pd.read_html(str(table))
table_df = pd.DataFrame(table_df[0])

table_df = table_df.drop(['Sort [1]', 'County Seat(s) [3]', 'Population(2010)', 'Land Areakm²', 'Land Areami²', 'Water Areakm²', 'Water Areami²', 'Total Areakm²', 'Total Areami²'], axis=1)
table_df = table_df.set_axis(['State', 'Geoid', 'County', 'Lat', 'Long'], axis=1)
table_df.Lat = table_df.Lat.str.replace('+', '').str.replace('°', '')
table_df.Long = table_df.Long.str.replace('–', '-').str.replace('°', '')

In [ ]:
table_df.County = table_df.County.str.replace(r'( \[.*?\])', '')

In [ ]:
stadiums_df = pd.read_excel('MLB Stadiums.xlsx').reset_index(drop=True)
stadiums_df

In [ ]:
min_distance = []
for i, j in zip(table_df.Lat, table_df.Long):
    county_coords = (float(i), float(j))
    distance_list = []
    for i, j in zip(stadiums_df.Latitude, stadiums_df.Longitude):
        stadium_coords = (float(i), float(j))
        distance_list.append(geopy.distance.distance(county_coords, stadium_coords).mi)
    min_distance.append(min(distance_list))

table_df['min_distance'] = min_distance

max_distance = []
for i, j in zip(table_df.Lat, table_df.Long):
    county_coords = (float(i), float(j))
    distance_list = []
    for i, j in zip(stadiums_df.Latitude, stadiums_df.Longitude):
        stadium_coords = (float(i), float(j))
        distance_list.append(geopy.distance.distance(county_coords, stadium_coords).mi)
    max_distance.append(max(distance_list))

table_df['max_distance'] = max_distance

In [ ]:
min_distance = None
closest_team = []
n = 0
for i, j in zip(table_df.Lat, table_df.Long):
    county_coords = (float(i), float(j))
    distance_list = []
    team_list = []
    for i, j, k in zip(stadiums_df.Latitude, stadiums_df.Longitude, stadiums_df.Team):
        stadium_coords = (float(i), float(j))
        distance_list.append(geopy.distance.distance(county_coords, stadium_coords).mi)
        team_list.append(k)
    min_distance = min(distance_list)
    min_index = distance_list.index(min_distance)
    if min_distance == table_df.at[n, 'min_distance']:
        closest_team.append(team_list[min_index])
    n += 1

table_df['closest_team'] = closest_team

max_distance = None
furthest_team = []
n = 0
for i, j in zip(table_df.Lat, table_df.Long):
    county_coords = (float(i), float(j))
    distance_list = []
    team_list = []
    for i, j, k in zip(stadiums_df.Latitude, stadiums_df.Longitude, stadiums_df.Team):
        stadium_coords = (float(i), float(j))
        distance_list.append(geopy.distance.distance(county_coords, stadium_coords).mi)
        team_list.append(k)
    max_distance = max(distance_list)
    max_index = distance_list.index(max_distance)
    if max_distance == table_df.at[n, 'max_distance']:
        furthest_team.append(team_list[max_index])
    n += 1

table_df['furthest_team'] = furthest_team

In [ ]:
table_df.to_excel('county_latlong.xlsx')

In [ ]:
table_df